**После завершения соревнования TODO:**

    Дочистить все данные, используя англоязычные адреса. 
    
    Можно использовать парсинг страниц категорий того, что находится рядом.
    
    Интересные фичи, которые предложили другие участники:
        признак  дистанция до центра города
        признак номер дома
        фича - mean_target по atm_group по регионам
        признак домножение всех признаков на atm_group
    
   **Почему было важно выяснить какому банку принадлежат определённые банкоматы:**
    
    По номеру банка atm_group проще найти банкомат в базе самого банка. А затем, по координатам или по описанию банкомата извлечь признаки локации (время работы, тип точки, название торговой сети, если магазин, доступные валютные операции и прочее) 
    
    Кто-то предложил вот такое: если в одной точке несколько банкоматов с разными таргетами, то можно усреднять значения прогноза, но это не фича, а патч оценщику

**ЗАМЕТКИ**
- Такое разделение на улицу и город нельзя использовать потому что в некоторых случаях теряем улицу/город. Т.о. что дальше делать с такой улицой или городом -- не ясно. 

```python
X['street'] = X.address.apply(lambda x: x[23:].strip())
X['street'].to_csv('temp.csv')

X.iloc[2871]['street']
'1 N. Novgorod'

X.iloc[2871].address, X.iloc[2871].address_rus

('VerkhnePecherskaya ul., 1 N. Novgorod ',
'Верхнепечёрская улица, 1, Нижний Новгород, Россия')
```

- Исправления и выбросы

    BLVD
    SLOB
    
    337 317-2 LENINA AV.          TOMSK        317-2 ЛЕНИНА  ПРОСПЕCТ           ТОМСК       
    Г. ЕЛЕC,  УЛИЦА  РАДИОТ       ЕЛЕC  
    2980 PAVLOVSK. TR., 251V       BARNAUL      ПАВЛОВСК. ТР., 251В       БАРНАУЛ 
    PROSPEK
    ,"NOVGOROD":"NIZHNIYNOVGOROD"
    ALEXANDRONE
    
    где есть шум: POL,     X[X['city'] == 'POL']

    ```python
    temp = 7076
    X.loc[temp].address_rus, X.loc[temp].address, X.loc[temp].city, X.loc[temp].lat, X.loc[temp].long
    ```
    8520	D. 25, PR-KT IM MUSY DZHA NABEREZHNYE	NaN	18.225063	-66.478912	==>> 55.740776, 52.406384
    440
    1389
    1338

- **Удалить из выборки, потому что не успел почистить**

    **index, id**
    
    1838 5188.0
    
    2196 7859.0	
    
    5667 5637.0	
    
    5759 6326.0
    
    6749 270.0
    
    7527 187.0
    
    8294 269.0
    
    
- **Проверить на равенство два DataFame**


```python
from pandas.util.testing import assert_frame_equal
assert_frame_equal(X.drop(['count_x', 'count_y'], axis=1), Xprev)
```


Какие имеем исходные знания:

- Популярность это функция от количества операций с этим банкоматом
- *atm_group* = 8083 -- группа банкоматов РОСБАНКА
- Id формировался на основе atm_group и уникального внутри каждой группы id банкомата. X.sort_values('id') даст Вам понимание его структуры.
- под банками партнерами подразумевается: а) у них беспроцентное снятие денег со счетов Росбанка

Александр Мамаев:
1. PSI таргета между train/test/private во всех попарных комбинациях меньше 0.05. 
2. Статистические тесты Манна-Уитни о равенстве средних не отвергают нулевые гипотезы для всех попарных комбинаций train/test/private.
3. Таргет обрезан сверху для удаления из выборки банкоматов, чья популярность не объясняется геолокацией банкомата. 
4. Разделение train/test/private. "Все это не так прозаично".

Egor Urvanov:
Немного покрутился с данными сегодня. Вот, что получается.

Всякие distance_i, var_i формируются а-ля cumsum и должны интерпретироваться как "некоторая характеристика для k ближайших банкоматов". Фичи в духе min, max и логарифмы от этих величин (+ `mean_log, var_log, median_log`) не выстрелили.


Александр Мамаев:
1. В системах банка адрес банкомата хранится в виде поля address. Поля address_rus, lat, long - восстанавливались на основе поля address. Для части банкоматов адрес и местоположение с точностью до дома установить не удалось. 

2. Но если взять поисковик на букву 'Г' (или любой другой до 'Я' включительно) и ввести в него значения поля address можно получить представление о той технологии с помощью которой поля заполнялись. 
Облегчить процесс представления может помочь "import selenium".


### Чистка данных
>
> Думал использовать *EllipticEnvelope* для поиска выбросов и автоматизировать, но времени разобраться с этим уже не было.
>
>```python
from sklearn.covariance import EllipticEnvelope
outliers_fraction = 0.15
elEn = EllipticEnvelope(contamination=outliers_fraction)
dots = X[X['city'] == city][['lat','long']].dropna()
elEn.fit(dots['lat','long'].values)
y_pred = elEn.fit(X).predict(X)
>```

sudo ./conda install -c conda-forge folium
sudo ./conda install -c conda-forge geocoder

import folium  - не оказался полезным
import geocoder   -- для быстрой проверки (исползовал на ранних этапах)

In [ ]:
https://files.scienceforum.ru/pdf/2012/1213.pdf
http://banki.ru
https://egrp365.ru/map/?kadnum=25:27:020102:983
                

In [ ]:
Александр Мамаев:
Решение должно включать:
1. Инструменты сбора признаков. (Функции, классы реализующие технологию обогащения данных, структуры данных в которых сохранялся результат работы.)
2. Ссылки на использованные источники внешних данных.
3. Код предобработки собранных признаков, код формирования обучающей выборки, код модели.
4. Лаконичные комментарии к коду рассчитанные на автора baseline'а :)

In [ ]:
RepeatedStratifiedKFold